### Test to save all estimators

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV, KFold, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.datasets import make_friedman1
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle
import json
import os
from sklearn.dummy import DummyRegressor
import seaborn as sns
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')
from utils import FriedmanDataset, ModelOptimizerFinal

: 

In [142]:
import itertools
json_file = "./test.json" # set path to save json-file 
# Data Creation
n_test= 100000
n_features = 10
# Random Search Parameters
n_folds = 5
n_iter= 20
scoring= 'neg_mean_squared_error' #Metriken anschauen
n_jobs= -1
# Anzahl Wiederholungen
n_repetitions = 10



# Define hyperparameter options
train_list = [200, 1000]
noise_list = [0, 5]
transformation_list = ['identity', 'sqrt']
group_size_list = [5, 10]


all_combinations = generate_hyperparameter_combinations_dict(n_train=train_list, 
                                                             noise=noise_list,
                                                             transformation=transformation_list, 
                                                             group_size=group_size_list
                                                             )
all_combinations


[{'n_train': 200, 'noise': 0, 'transformation': 'identity', 'group_size': 5},
 {'n_train': 200, 'noise': 0, 'transformation': 'identity', 'group_size': 10},
 {'n_train': 200, 'noise': 0, 'transformation': 'sqrt', 'group_size': 5},
 {'n_train': 200, 'noise': 0, 'transformation': 'sqrt', 'group_size': 10},
 {'n_train': 200, 'noise': 5, 'transformation': 'identity', 'group_size': 5},
 {'n_train': 200, 'noise': 5, 'transformation': 'identity', 'group_size': 10},
 {'n_train': 200, 'noise': 5, 'transformation': 'sqrt', 'group_size': 5},
 {'n_train': 200, 'noise': 5, 'transformation': 'sqrt', 'group_size': 10},
 {'n_train': 1000, 'noise': 0, 'transformation': 'identity', 'group_size': 5},
 {'n_train': 1000, 'noise': 0, 'transformation': 'identity', 'group_size': 10},
 {'n_train': 1000, 'noise': 0, 'transformation': 'sqrt', 'group_size': 5},
 {'n_train': 1000, 'noise': 0, 'transformation': 'sqrt', 'group_size': 10},
 {'n_train': 1000, 'noise': 5, 'transformation': 'identity', 'group_size': 5},

In [144]:
# Display the combined hyperparameter dictionaries
for hyperparameter_dict in all_combinations:
    print("1", hyperparameter_dict['n_train'])

1 200
1 200
1 200
1 200
1 200
1 200
1 200
1 200
1 1000
1 1000
1 1000
1 1000
1 1000
1 1000
1 1000
1 1000


In [139]:
import itertools

def generate_hyperparameter_combinations_dict(**hyperparameters):
    """
    Generate all possible combinations of hyperparameters.

    Parameters:
    **hyperparameters: Variable keyword arguments where the key is the hyperparameter name and the value is an iterable of options (best: list).

    Returns:
    List of dictionaries, where each dictionary represents a combination of hyperparameters.
    """
    hyperparameter_names = hyperparameters.keys()
    all_hyperparameter_combinations = list(itertools.product(*hyperparameters.values()))

    all_hyperparameter_dicts = []
    for combination in all_hyperparameter_combinations:
        hyperparameter_dict = dict(zip(hyperparameter_names, combination))
        all_hyperparameter_dicts.append(hyperparameter_dict)

    return all_hyperparameter_dicts

In [133]:
n_train = [200, 1000]
n_features = 10
FD_noise = [0, 5]
transformation = ['identity', 'sqrt']
group_size = [5, 10]
n_folds = 5
n_iter = 200
itertools.combinations(n_train, FD_noise, transformation, group_size)

In [88]:
json_file = "./test.json" # set path to save json-file 
# Data Creation
n_train= 200
n_test= 100000
n_features = 10
FD_noise= 0
transformation='sqrt'
seed=567

# Stratification
group_size = 10

# RandomSearchCV
n_folds = 5
n_iter= 20
n_repetitions= 10
scoring= 'neg_mean_squared_error' #Metriken anschauen
n_jobs= -1

RF_param_grid = {
    'min_samples_split': np.arange(2, 21),
    'min_samples_leaf': np.arange(1, 21),
    'max_features': np.arange(1, n_features + 1) #@nadja is that right?
}

# Initalize Model
modelOptimizer = ModelOptimizerFinal(RandomForestRegressor(random_state=seed), 
                                RF_param_grid,
                                random_state=seed)

model = RandomForestRegressor(n_estimators=1000, random_state=seed) 
# Save Parameters in a dictionary
params = {'model': modelOptimizer.model.__class__.__name__,
          'n_train': n_train,
          'n_test': n_test,
          'n_features': n_features,
          'FD_noise': FD_noise,
          'transformation': transformation,
          'n_groups': group_size,
          'n_folds': n_folds,
          'n_iter': n_iter,
          'n_repetitions': n_repetitions,
          'scoring': scoring, 
          'n_jobs': n_jobs,
          'json_file': json_file}
params

{'model': 'RandomForestRegressor',
 'n_train': 200,
 'n_test': 100000,
 'n_features': 10,
 'FD_noise': 0,
 'transformation': 'sqrt',
 'n_groups': 10,
 'n_folds': 5,
 'n_iter': 20,
 'n_repetitions': 10,
 'scoring': 'neg_mean_squared_error',
 'n_jobs': -1,
 'json_file': './test.json'}

In [59]:
X_test, y_test = generate_friedman1(n_samples=n_test,
                                                     n_features=n_features,
                                                     noise=FD_noise,
                                                     random_state=1718)



In [60]:
X_train, y_train = generate_friedman1(n_samples=n_train,
                                                        n_features=n_features,
                                                        noise=FD_noise,
                                                        random_state=random_state,
                                                        transformation=transformation)
            

In [61]:
# Check for NaN values in the data
if np.isnan(y_train).any() or np.isnan(y_test).any(): 
                X_train, y_train = make_friedman1(n_samples=n_train,
                                    n_features=n_features, 
                                    noise=noise, 
                                    random_state=random_states[repetition])
                X_test, y_test = make_friedman1(n_samples=n_test,
                                    n_features=n_features,
                                    noise=noise,
                                    random_state=1718)
                min_val = min(y_train.min(), y_test.min())
                # @Anne: Noch absprechen... +1 weil es sonst Skala durch Werte [0, 1] hauptsächlich bei y_test größer wird.           
                y_train = y_train + abs(min_val) + 1
                y_test = y_test + abs(min_val) + 1
                if transformation=='identity':
                    pass
                elif transformation == 'log':
                    y_train = np.log(y_train)
                    y_test = np.log(y_test)
                elif transformation == 'sqrt':
                    y_train = np.sqrt(y_train)
                    y_test = np.sqrt(y_test)

In [70]:
import time

In [71]:
cv_splits = n_folds
random_state = 45
start_time = time.time()
random_search = RandomizedSearchCV(estimator=model,
                                           param_distributions=RF_param_grid,
                                           n_iter=n_iter,
                                           cv=cv_splits,
                                           scoring=scoring,
                                           n_jobs=n_jobs,
                                           random_state=random_state,
                                           verbose=10)
random_search.fit(X_train, y_train)
end_time = time.time()

Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [72]:
running_time = end_time - start_time
running_time

75.47038507461548

In [73]:
results_unstratified = random_search.cv_results_

In [76]:
results_unstratified

{'mean_fit_time': array([5.00990644, 5.65276647, 5.11320963, 5.92497511, 5.98721724,
        4.36695023, 4.61709428, 3.67889953, 4.29646788, 3.97168398,
        5.54078302, 4.33252978, 4.33474655, 4.59259844, 4.80369287,
        4.32539926, 6.72153368, 5.40253696, 6.19697189, 4.97057409]),
 'std_fit_time': array([0.17157758, 0.27067507, 0.12507472, 0.12701229, 0.14979604,
        0.191261  , 0.24553393, 0.24391681, 0.10954427, 0.09218986,
        0.18261605, 0.13713153, 0.22126044, 0.26291373, 0.25915343,
        0.1929958 , 0.24840166, 0.29750458, 0.14446448, 0.56954533]),
 'mean_score_time': array([0.14109292, 0.15246305, 0.14343815, 0.12090912, 0.13220224,
        0.12727895, 0.12575464, 0.14821725, 0.14123712, 0.12665305,
        0.13337579, 0.13372936, 0.14349365, 0.13539653, 0.13778725,
        0.1336915 , 0.13298821, 0.14719005, 0.11945004, 0.1032022 ]),
 'std_score_time': array([0.02394724, 0.00612765, 0.01504996, 0.01544715, 0.01415603,
        0.01448361, 0.01802225, 0.019257

In [67]:
results_unstratified['param_min_samples_split']

masked_array(data=[19, 13, 16, 18, 8, 20, 18, 13, 12, 18, 20, 17, 9, 20,
                   5, 20, 15, 2, 15, 6],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False],
       fill_value='?',
            dtype=object)

In [147]:
with open("./seeds_available.json", "r") as f:
    seeds_available = json.load(f)

seeds_available

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [148]:
import json

# Path to the original file
path_to_seeds_original = './seeds_available.json'

# Read the content of the original JSON file
with open(path_to_seeds_original, 'r') as file:
    try:
        seeds_available = json.load(file)
    except json.JSONDecodeError:
        print("Error decoding JSON. The file might be empty or not properly formatted.")
        seeds_available = []


In [149]:

# Path to the new file
path_to_seeds_new = './seeds_available_fixed.json'

# Write the content to the new file in the correct format
with open(path_to_seeds_new, 'w') as file:
    json.dump(seeds_available, file, indent=4)

print(f"File successfully rewritten at: {path_to_seeds_new}")

File successfully rewritten at: ./seeds_available_fixed.json


In [56]:
# Get the best model from the RandomizedSearchCV
best_model = random_search.best_estimator_

results_unstratified = random_search.cv_results_
mse_list_unstratified = []

# Iterate through the results and make predictions for each iteration
for index, (mean_score, params) in enumerate(zip(results_unstratified["mean_test_score"], results_unstratified["params"])):

    best_model.set_params(**params)
    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    mse_list_unstratified.append(mse)

    print(f"Iteration {index} - Mean Squared Error: {mse}, Parameters: {params}")



Iteration - Mean Squared Error: 0.1820069225212941, Parameters: {'min_samples_split': 19, 'min_samples_leaf': 19, 'max_features': 8}
Iteration - Mean Squared Error: 0.14134281792383813, Parameters: {'min_samples_split': 13, 'min_samples_leaf': 10, 'max_features': 7}
Iteration - Mean Squared Error: 0.13979710330524364, Parameters: {'min_samples_split': 16, 'min_samples_leaf': 8, 'max_features': 5}
Iteration - Mean Squared Error: 0.12663663941987063, Parameters: {'min_samples_split': 18, 'min_samples_leaf': 1, 'max_features': 6}
Iteration - Mean Squared Error: 0.14083267857546644, Parameters: {'min_samples_split': 8, 'min_samples_leaf': 11, 'max_features': 10}
Iteration - Mean Squared Error: 0.235536751927918, Parameters: {'min_samples_split': 20, 'min_samples_leaf': 12, 'max_features': 2}
Iteration - Mean Squared Error: 0.18506289664078565, Parameters: {'min_samples_split': 18, 'min_samples_leaf': 20, 'max_features': 10}
Iteration - Mean Squared Error: 0.28783470953072376, Parameters: {

In [57]:
cv_splits = create_cont_folds(y=y_train, n_folds=n_folds, n_groups=20, seed=random_state)
random_state = 45
random_search = RandomizedSearchCV(estimator=model,
                                           param_distributions=RF_param_grid,
                                           n_iter=n_iter,
                                           cv=cv_splits,
                                           scoring=scoring,
                                           n_jobs=n_jobs,
                                           random_state=random_state,
                                           verbose=10)
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


RandomizedSearchCV(cv=[([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 17, 18,
                         19, 20, 21, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, ...],
                        array([ 10,  12,  16,  22,  23,  24,  32,  41,  45,  49,  50,  51,  66,
        67,  73,  74,  83,  85,  90,  91, 101, 102, 107, 110, 111, 124,
       129, 135, 137, 141, 143, 144, 155, 159, 165, 173, 177, 178, 189,
       197], dtype=int64)),
                       ([2, 3, 5, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20,
                         22, 23, 24, 27, 29, 30, 31, 32, 34, 35, 36, 37,...
                   estimator=RandomForestRegressor(n_estimators=1000,
                                                   random_state=567),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'max_features': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'min_samples_leaf': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20]),
                                        'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20])},
                   random_state=45, scoring='neg_mean_squared_error',
                   verbose=10)

In [85]:
results_stratified

{'mean_fit_time': array([3.80345759, 4.34663281, 3.98740835, 4.64461303, 5.00456128,
        3.86503544, 4.68733215, 3.56064806, 3.50343966, 3.39808583,
        4.58296652, 3.47389059, 3.48993802, 3.7865128 , 3.84228473,
        3.46228442, 5.5025279 , 4.44740472, 4.99406586, 4.17634463]),
 'std_fit_time': array([0.11953779, 0.18809168, 0.15560952, 0.1235797 , 0.20425711,
        0.41863452, 0.31302975, 0.05441163, 0.06308479, 0.07235807,
        0.07995153, 0.02154357, 0.05660094, 0.07495044, 0.0929972 ,
        0.05607092, 0.14094468, 0.14945619, 0.13739465, 0.23138328]),
 'mean_score_time': array([0.10980549, 0.11022758, 0.11578217, 0.11509714, 0.13213353,
        0.12587032, 0.10664091, 0.11917171, 0.11752744, 0.11994386,
        0.11132212, 0.12651081, 0.12832785, 0.11984725, 0.10637932,
        0.11040831, 0.11430936, 0.1167923 , 0.11419854, 0.08524981]),
 'std_score_time': array([0.01022314, 0.00858472, 0.01319274, 0.00534519, 0.01906321,
        0.01495005, 0.01504643, 0.009022

In [63]:
for index, (mean_score, params) in enumerate(zip(results_stratified["mean_test_score"], results_stratified["params"])):
    print(f"Iteration {index} - Mean Test Score: {mean_score}, Parameters: {params}")

Iteration 0 - Mean Test Score: -0.21967496002005796, Parameters: {'min_samples_split': 19, 'min_samples_leaf': 19, 'max_features': 8}
Iteration 1 - Mean Test Score: -0.16507660123457998, Parameters: {'min_samples_split': 13, 'min_samples_leaf': 10, 'max_features': 7}
Iteration 2 - Mean Test Score: -0.1604424932683084, Parameters: {'min_samples_split': 16, 'min_samples_leaf': 8, 'max_features': 5}
Iteration 3 - Mean Test Score: -0.1506118625798829, Parameters: {'min_samples_split': 18, 'min_samples_leaf': 1, 'max_features': 6}
Iteration 4 - Mean Test Score: -0.16990598044204633, Parameters: {'min_samples_split': 8, 'min_samples_leaf': 11, 'max_features': 10}
Iteration 5 - Mean Test Score: -0.23909437831753796, Parameters: {'min_samples_split': 20, 'min_samples_leaf': 12, 'max_features': 2}
Iteration 6 - Mean Test Score: -0.23143249183877268, Parameters: {'min_samples_split': 18, 'min_samples_leaf': 20, 'max_features': 10}
Iteration 7 - Mean Test Score: -0.27924755819356184, Parameters: 

In [89]:
for index, (mean_score, params) in enumerate(zip(results_unstratified['mean_test_score'],
                                                results_unstratified["params"])):
    print(f"Iteration {index} - Mean Test Score: {mean_score}, Parameters: {params}")

Iteration 0 - Mean Test Score: -0.21089212802635998, Parameters: {'min_samples_split': 19, 'min_samples_leaf': 19, 'max_features': 8}
Iteration 1 - Mean Test Score: -0.16126511127811952, Parameters: {'min_samples_split': 13, 'min_samples_leaf': 10, 'max_features': 7}
Iteration 2 - Mean Test Score: -0.15773293251844192, Parameters: {'min_samples_split': 16, 'min_samples_leaf': 8, 'max_features': 5}
Iteration 3 - Mean Test Score: -0.14563733723230055, Parameters: {'min_samples_split': 18, 'min_samples_leaf': 1, 'max_features': 6}
Iteration 4 - Mean Test Score: -0.16567433663782424, Parameters: {'min_samples_split': 8, 'min_samples_leaf': 11, 'max_features': 10}
Iteration 5 - Mean Test Score: -0.24255398936875813, Parameters: {'min_samples_split': 20, 'min_samples_leaf': 12, 'max_features': 2}
Iteration 6 - Mean Test Score: -0.21956869342455643, Parameters: {'min_samples_split': 18, 'min_samples_leaf': 20, 'max_features': 10}
Iteration 7 - Mean Test Score: -0.28548234553308954, Parameters

In [88]:
print(round(np.var(results_unstratified['std_test_score']), 6), float(round(np.var(results_stratified['std_test_score']), 6)))

0.0002 4.1e-05


choplip (parallelisierung)

In [65]:
# Get the best model from the RandomizedSearchCV
best_model = random_search.best_estimator_

results_stratified = random_search.cv_results_
mse_list_stratified = []

# Iterate through the results and make predictions for each iteration
for index, (mean_score, params) in enumerate(zip(results_stratified["mean_test_score"], results_stratified["params"])):
   
    best_model.set_params(**params)
    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mse_list_stratified.append(mse)
    print(f"Iteration {index} - MSE: {mse}, Parameters: {params}")

Iteration 0 - Mean Test Score: 0.1820069225212941, Parameters: {'min_samples_split': 19, 'min_samples_leaf': 19, 'max_features': 8}
Iteration 1 - Mean Test Score: 0.14134281792383813, Parameters: {'min_samples_split': 13, 'min_samples_leaf': 10, 'max_features': 7}
Iteration 2 - Mean Test Score: 0.13979710330524364, Parameters: {'min_samples_split': 16, 'min_samples_leaf': 8, 'max_features': 5}
Iteration 3 - Mean Test Score: 0.12663663941987063, Parameters: {'min_samples_split': 18, 'min_samples_leaf': 1, 'max_features': 6}
Iteration 4 - Mean Test Score: 0.14083267857546644, Parameters: {'min_samples_split': 8, 'min_samples_leaf': 11, 'max_features': 10}
Iteration 5 - Mean Test Score: 0.235536751927918, Parameters: {'min_samples_split': 20, 'min_samples_leaf': 12, 'max_features': 2}
Iteration 6 - Mean Test Score: 0.18506289664078565, Parameters: {'min_samples_split': 18, 'min_samples_leaf': 20, 'max_features': 10}
Iteration 7 - Mean Test Score: 0.28783470953072376, Parameters: {'min_sam

In [59]:
mse_list_stratified==mse_list_unstratified

True

In [60]:
results_stratified["params"] == results_unstratified["params"]

True

In [54]:
def perform_optimization(X_train, 
                              y_train, 
                              X_test, 
                              y_test, 
                              cv, 
                              n_groups, 
                              scoring, 
                              n_jobs, 
                              n_iter, 
                              random_state,
                              stratified):
        '''
        Function to perform the optimization.
        Inputs:
            the same as in optimize function
            stratified: whether to use stratified k-fold or not
        Outputs:
            evaluation_results: the evaluation results in a dictionary
            best_params: the best parameters in a dictionary
        '''
        if stratified:
            cv_splits = create_cont_folds(y_train, n_folds=cv, n_groups=n_groups)
            output_text = 'Stratified Split Cross-validation'
        else:
            cv_splits = cv
            output_text = 'Random Split Cross-validation'
        
        start_time = time.time()
        random_search = RandomizedSearchCV(estimator=model,
                                           param_distributions=RF_param_grid,
                                           n_iter=n_iter,
                                           cv=cv_splits,
                                           scoring=scoring,
                                           n_jobs=n_jobs,
                                           random_state=random_state)
        random_search.fit(X_train, y_train)
        end_time = time.time()
        running_time = end_time - start_time
        print("Best Parameters:", random_search.best_params_)

        # Evaluate the model
        evaluation_results = evaluate_rf(random_search, X_train, X_test, y_train, y_test)
        print("Evaluation Results of", output_text, ': ', evaluation_results)
        print('running_time: ', round(running_time/60, 2), ' min')
        
        return evaluation_results, random_search.best_params_, running_time

In [55]:

def create_cont_folds(y, 
                          n_folds=5, 
                          n_groups=5, 
                          seed=1):
        '''
        Function to create continuous folds.
        Inputs:
            y: the target variable
            n_folds: the number of folds
            n_groups: the number of groups (based on quantiles)
            seed: the seed to be used
        Outputs:
            cv_splits: the indices for the folds
        '''
        # create StratifiedKFold like for classification
        skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)

        # create groups in y with pd.qcut: quantile-based discretization 
        y_grouped = pd.qcut(y, n_groups, labels=False)

        # create fold numbers    
        fold_nums = np.zeros(len(y))
        #split(X, y[, groups]): Generate indices to split data into training and test set
        for fold_no, (t, v) in enumerate(skf.split(y_grouped, y_grouped)): #@Nadja: unabhängig von n_folds? n_folds = fol_no, test_data_size = N/n_folds
            fold_nums[v] = fold_no

        cv_splits = []

        # iterate over folds and creat train and test indices for each fold
        for i in range(n_folds):
            test_indices = np.argwhere(fold_nums==i).flatten()
            train_indices = list(set(range(len(y_grouped))) - set(test_indices))
            cv_splits.append((train_indices, test_indices))

        return cv_splits

def evaluate_rf(model, X_train, X_test, y_train, y_test):
        '''
        Function to evaluate the model.
        Inputs:
            model: the model to be evaluated
            X_train, X_test, y_train, y_test: the train and test data
        Outputs:
            dictionary with the evaluation results (R2, MSE, MAE)
        '''
        model=model.best_estimator_
        # @Anne: This somehow also does not work, do not know why.
        #best_score = model.best_score_

        train_r2, test_r2 = round(model.score(X_train, y_train), 4), round(model.score(X_test, y_test), 4)
        y_train_pred, y_test_pred = model.predict(X_train), model.predict(X_test)
        train_mse, test_mse = round(mean_squared_error(y_train, y_train_pred), 4), round(mean_squared_error(y_test, y_test_pred), 4)
        train_mae, test_mae = round(mean_absolute_error(y_train, y_train_pred), 4), round(mean_absolute_error(y_test, y_test_pred), 4)
        return {'train r2': train_r2, 
                'test r2': test_r2, 
                'train mse': train_mse,
                'test mse': test_mse,
                'train mae': train_mae,
                'test mae': test_mae}
        
     
def _convert_numpy_types(obj):
        '''
        Function to convert numpy types.
        Inputs:
            obj: the object to be converted
        Outputs:
            the converted object
        '''
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, np.int32):
            return int(obj)
        elif isinstance(obj, np.int64):
            return int(obj)
        elif isinstance(obj, (list, tuple)):
            return [_convert_numpy_types(item) for item in obj]
        elif isinstance(obj, dict):
            return {key: _convert_numpy_types(value) for key, value in obj.items()}
        else:
            return obj

In [56]:
def generate_friedman1(n_samples, n_features, noise, random_state, transformation='identity'):
        '''
        Function to generate dataset according to Friedman1.
        Inputs:
            n_samples: number of data points
            n_features: number of features (have to be at least 5)
            noise: The standard deviation of the gaussian noise applied to the output.
            random_state: to repreoduce dataset
        Outputs:
            features: array
            y: array

        '''
        features, y = make_friedman1(n_samples=n_samples, 
                                    n_features=n_features, 
                                    noise=noise, 
                                    random_state=random_state)
        if transformation=='identity':
            pass
        elif transformation == 'log':
            y = np.log(y)
             
        elif transformation == 'sqrt':
            y = np.sqrt(y)
             
        else:
            raise ValueError('Transformation not implemented.')

        return features, y

In [57]:
def optimize(    params,
                 data='friedman',
                 random_states=None):
        '''
        Function to optimize the model.
        Inputs:
            X_train, X_test, y_train, y_test: the train and test data
            cv: the number of folds
            n_groups: the number of groups (based on quantiles)
            scoring: the scoring to be used
            n_jobs: the number of jobs to be used
            n_iter: the number of iterations
            ROOT_PATH: the root path to be used (results are stored in JSON file)
            transformation: the transformation of target to be applied
        Outputs:
            unstratified_results: the results of the unstratified cross-validation
            stratified_results: the results of the stratified cross-validation
        Important: The results are stored in a JSON file. Initialize a new file with an empty list as content.
        '''
                                                     
        
        # get parameters from params dictionary
        n_train = params['n_train']
        n_test = params['n_test']
        n_features = params['n_features']
        noise = params['FD_noise']
        transformation = params['transformation']
        n_folds= params['n_folds']
        n_groups = params['n_groups']
        scoring = params['scoring']
        n_jobs = params['n_jobs']
        n_iter = params['n_iter']
        n_repetitions = params['n_repetitions']
        json_file = params['json_file']

        if data == 'friedman':
            # maybe implement accessing and generating the data nicer
            X_test, y_test = generate_friedman1(n_samples=n_test,
                                                     n_features=n_features,
                                                     noise=noise,
                                                     random_state=1718,
                                                     transformation=transformation)
        # @Anne und auch @Nadja: macht das Sinn?
        if not isinstance(random_states, list):
            random.seed(seed)
            random_states = random.sample(range(1, 10000), n_repetitions)

        else:
            random_states = random_states[:n_repetitions]

        print("RandomizesdSearchCV with params n_folds =",
            n_folds, ", ngroups: ", n_groups, ", scoring: ",scoring, ", n_jobs: ",n_jobs,
            ", n_iter: ", n_iter, " and save to  ", json_file, "\n")
        
        all_results = {}
        all_results_stratified = {}

        initialization = {
            'model_info': params,
            'seed': seed
        }
        for repetition in range(n_repetitions):
            if data == 'friedman':
                X_train, y_train = generate_friedman1(n_samples=n_train,
                                                        n_features=n_features,
                                                        noise=noise,
                                                        random_state=random_states[repetition],
                                                        transformation=transformation)
            # Check for NaN values in the data
            if np.isnan(y_train).any() or np.isnan(y_test).any(): 
                X_train, y_train = make_friedman1(n_samples=n_train,
                                    n_features=n_features, 
                                    noise=noise, 
                                    random_state=random_states[repetition])
                X_test, y_test = make_friedman1(n_samples=n_test,
                                    n_features=n_features,
                                    noise=noise,
                                    random_state=1718)
                min_val = min(y_train.min(), y_test.min())
                # @Anne: Noch absprechen... +1 weil es sonst Skala durch Werte [0, 1] hauptsächlich bei y_test größer wird.           
                y_train = y_train + abs(min_val) + 1
                y_test = y_test + abs(min_val) + 1
                if transformation=='identity':
                    pass
                elif transformation == 'log':
                    y_train = np.log(y_train)
                    y_test = np.log(y_test)
                elif transformation == 'sqrt':
                    y_train = np.sqrt(y_train)
                    y_test = np.sqrt(y_test)

            # Perform optimization with unstratified cross-validation
            unstratified_results, unstratified_params, unstratified_running_time = perform_optimization(X_train, 
                                                            y_train, 
                                                            X_test,
                                                            y_test,
                                                            n_folds, 
                                                            n_groups,
                                                            scoring, 
                                                            n_jobs, 
                                                            n_iter, 
                                                            random_states[repetition],
                                                            stratified=False)
            all_results.update({f'Repetition {repetition}': unstratified_results})

            # Perform optimization with stratified cross-validation
            stratified_results, stratified_params, stratified_running_time = perform_optimization(X_train, 
                                                            y_train, 
                                                            X_test,
                                                            y_test,
                                                            n_folds, 
                                                            n_groups, 
                                                            scoring, 
                                                            n_jobs, 
                                                            n_iter, 
                                                            random_states[repetition],
                                                            stratified=True)
            all_results_stratified.update({f'Repetition {repetition}': stratified_results}) # @Anne: check ich nicht ganz, was das macht

            if unstratified_params == stratified_params:
                hyperparameters_same = True
            else:
                hyperparameters_same = False
            
            # Save results and parameters to a file
            results = {
                'repetition': repetition,
                'random_state': random_states[repetition],
                'hyperparameters_same': hyperparameters_same,
                'unstratified_params':unstratified_params,
                'stratified_params': stratified_params,
                'unstratified_results': unstratified_results,
                'stratified_results': stratified_results,
                'unstratified_running_time': round(unstratified_running_time,2), 
                'stratified_running_time': round(stratified_running_time, 2)
            }

            initialization.update(results)
            # Load existing data or create an empty list
            with open(json_file, 'r') as file:
                existing_data = json.load(file)

            # Append the new results dictionary to the existing data
            existing_data.append(initialization)

            # Write the updated data back to the JSON file
            with open(json_file, 'w') as file:
                json.dump(existing_data, file, indent=4, default=_convert_numpy_types)

        return all_results, all_results_stratified